In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import statsmodels.api as sm
import scipy.stats as st 
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_absolute_percentage_error,mean_absolute_error, mean_squared_error

load data

In [ ]:
hc_train_f = pd.read_csv('data/F_na_data_train.csv')['Participant ID'].to_list()
hc_val_f = pd.read_csv('data/F_na_data_val.csv')['Participant ID'].to_list()
hc_test_f = pd.read_csv('data/F_na_data_test.csv')['Participant ID'].to_list()
hc_list_f = [*hc_train_f, *hc_val_f, *hc_test_f]
hc_train_m = pd.read_csv('data/M_na_data_train.csv')['Participant ID']
hc_val_m = pd.read_csv('data/M_na_data_val.csv')['Participant ID']
hc_test_m = pd.read_csv('data/M_na_data_test.csv')['Participant ID']
hc_list_m = [*hc_train_m, *hc_val_m, *hc_test_m]
label = pd.read_csv('/home/jielian/DXA_BA/results/center_all.csv')
# label = pd.read_csv('/home/jielian/DXA_BA/results/center_all_0406.csv')
tradition = pd.read_csv('/home/jielian/DXA_BA/data/tradition.csv')
label = label.merge(tradition, on = ['Participant ID'], how = 'left')
label_f = label[label['Sex']== 'Female']
label_m = label[label['Sex']== 'Male']
f_data = pd.read_csv('data/feature_df.csv')[['Participant ID','Total BMD (bone mineral density) T-score | Instance 2','VAT_Rate', 'lean_Rate']]
f_data.rename(columns={'Total BMD (bone mineral density) T-score | Instance 2':'BMD T-score'}, inplace=True)

remove NA data

In [ ]:
na_patients= pd.read_csv('data/na_hc.csv')['Remove'].to_list()
label_f = label_f[~label_f['Participant ID'].isin(na_patients)]
label_m = label_m[~label_m['Participant ID'].isin(na_patients)]

In [ ]:
label_f = label_f.merge(f_data, on = ['Participant ID'], how ='inner')
label_m = label_m.merge(f_data, on = ['Participant ID'], how ='inner')
label_f.rename(columns={'Age when attended assessment centre | Instance 2':'CA', 'B-Age':'BA'}, inplace=True)
label_m.rename(columns={'Age when attended assessment centre | Instance 2':'CA', 'B-Age':'BA'}, inplace=True)

In [ ]:
f_train_y = np.array(label_f[label_f['Participant ID'].isin(hc_train_f)]['CA'])
f_train_pred = np.array(label_f[label_f['Participant ID'].isin(hc_train_f)]['BA']).reshape(-1,1)
calibrator = LinearRegression()
calibrator.fit(f_train_pred, f_train_y)
y_calibrated_train = calibrator.predict(f_train_pred)
mse_uncalibrated = mean_squared_error(f_train_y, f_train_pred)
print('Train')
print(mse_uncalibrated)
mse_calibrated = mean_squared_error(f_train_y, y_calibrated_train)
print(mse_calibrated)
print('test')
f_test_y = np.array(label_f[label_f['Participant ID'].isin(hc_test_f)]['CA'])
f_test_pred = np.array(label_f[label_f['Participant ID'].isin(hc_test_f)]['BA']).reshape(-1,1)
y_calibrated_test = calibrator.predict(f_test_pred)
mse_uncalibrated = mean_squared_error(f_test_y, f_test_pred)
print(mse_uncalibrated)
mse_calibrated = mean_squared_error(f_test_y, y_calibrated_test)
print(mse_calibrated)

label_f['C_BA'] = calibrator.predict(np.array(label_f['BA']).reshape(-1,1))

In [ ]:
m_train_y = np.array(label_m[label_m['Participant ID'].isin(hc_train_m)]['CA'])
m_train_pred = np.array(label_m[label_m['Participant ID'].isin(hc_train_m)]['BA']).reshape(-1,1)
calibrator = LinearRegression()
calibrator.fit(m_train_pred, m_train_y)
y_calibrated_train = calibrator.predict(m_train_pred)
mse_uncalibrated = mean_squared_error(m_train_y, m_train_pred)
print('Train')
print(mse_uncalibrated)
mse_calibrated = mean_squared_error(m_train_y, y_calibrated_train)
print(mse_calibrated)
print('test')
m_test_y = np.array(label_m[label_m['Participant ID'].isin(hc_test_m)]['CA'])
m_test_pred = np.array(label_m[label_m['Participant ID'].isin(hc_test_m)]['BA']).reshape(-1,1)
y_calibrated_test = calibrator.predict(m_test_pred)
mse_uncalibrated = mean_squared_error(m_test_y, m_test_pred)
print(mse_uncalibrated)
mse_calibrated = mean_squared_error(m_test_y, y_calibrated_test)
print(mse_calibrated)
label_m['C_BA'] = calibrator.predict(np.array(label_m['BA']).reshape(-1,1))


In [ ]:
def BC_label(df, hc_list, vatlow, vatup, leanlow, leanup):
    new_list = []
    new_list_vat = []

    for i in range(len(df)):
        l = df.loc[i]
        if l['Participant ID'] in hc_list:
            new_list.append('Normal')
            new_list_vat.append('Normal')
        else:
            ints_cols = l[['T2D Label', 'Hypertension Label', 'MACE Label', 'ASCVD Label']].to_list()
            # print(ints_cols)
            if ints_cols == ['HC', 'HC', 'HC', 'HC']:
                new_list.append('HC')
                
                vat = l['VAT_Rate']
                lean = l['lean_Rate']

                if vat < vatlow and lean > leanup:
                    new_list_vat.append('Hypernormal')
                elif vat > vatup and lean < leanlow:
                    new_list_vat.append('Suboptimal')
                else:
                    new_list_vat.append('Other Normal')
                
            else:
                ints_cols = l[['T2D Label', 'Hypertension Label', 'MACE Label', 'ASCVD Label']].to_list()
                # print(ints_cols)
                if 'Before' in ints_cols and 'After' not in ints_cols:
                    new_list.append('Pre-existing Disease')
                    new_list_vat.append('Pre-existing Disease')
                elif 'Before' in ints_cols and 'After' in ints_cols:
                    new_list.append('Post-DXA and Pre-Disease')
                    new_list_vat.append('Post-DXA and Pre-Disease')
                else:
                    new_list.append('Post-DXA Disease')
                    new_list_vat.append('Post-DXA Disease')
    return new_list, new_list_vat


In [ ]:
f_thred1 = 0.30302540571529024
f_thred2 = 1.573286500910589
f_thred3 = 54.98972216226349
f_thred4 = 69.1795768225732
m_thred1 = 0.8764653962601622
m_thred2 = 2.6256245114927093
m_thred3 = 65.11126004803509
m_thred4 = 77.83297203021218
label_f['BC_label'], label_f['VAT_label']= BC_label(label_f, hc_list_f, f_thred1, f_thred2,f_thred3, f_thred4)
label_m['BC_label'], label_m['VAT_label'] = BC_label(label_m, hc_list_m, m_thred1, m_thred2, m_thred3, m_thred4)
print('===================================')
print(label_f['BC_label'].value_counts())
print('-------------------------------')
print(label_f['VAT_label'].value_counts())
print('-------------------------------')
# print(label_f['Lean_label'].value_counts())
print('===================================')
print(label_m['BC_label'].value_counts())
print('-------------------------------')
print(label_m['VAT_label'].value_counts())
print('-------------------------------')

In [ ]:
# def table_generator(label_f):
#     disease_b = label_f[label_f['BC_label'].isin(['Pre-existing Disease', 'Post-DXA and Pre-Disease'])]
#     disease_a = label_f[label_f['BC_label'].isin(['Post-DXA Disease','Post-DXA and Pre-Disease'])]
#     print(len(disease_b))
#     print(len(disease_a))

#     nd = label_f[label_f['BC_label'] =='HC']
#     normal = label_f[label_f['BC_label'] =='Normal']
#     df = pd.DataFrame()

#     df['n'] = ['Normal Reference '+'(n= '+str(len(normal)) +')','Non-Normal '+'(n= '+str(len(nd))+')', 'Pre-existing disease '+'(n= '+str(len(disease_b))+')', 'Post-DXA disease '+'(n= '+str(len(disease_a))+')']
#     df['Age Mean'] = [normal['CA'].mean(), nd['CA'].mean(), disease_b['CA'].mean(), disease_a['CA'].mean()]
#     df['Age STD'] = [normal['CA'].std(), nd['CA'].std(), disease_b['CA'].std(), disease_a['CA'].std()]
#     f_statistic1, p_value1 = st.f_oneway(normal['CA'], nd['CA'], disease_b['CA'], disease_a['CA'])
#     df['Age statisctis'] = [p_value1,0,0,0]
#     df['VAT Mean'] = [normal['VAT_Rate'].mean(), nd['VAT_Rate'].mean(), disease_b['VAT_Rate'].mean(), disease_a['VAT_Rate'].mean()]
#     df['VAT STD'] = [normal['VAT_Rate'].std(), nd['VAT_Rate'].std(), disease_b['VAT_Rate'].std() , disease_a['VAT_Rate'].std()]
#     f_statistic2, p_value2 = st.f_oneway(normal['VAT_Rate'], nd['VAT_Rate'], disease_b['VAT_Rate'].dropna(),  disease_a['VAT_Rate'].dropna() )
#     df['VAT statisctis'] = [p_value2, 0,0,0]
#     df['Lean Mean'] = [normal['lean_Rate'].mean(), nd['lean_Rate'].mean(), disease_b['lean_Rate'].mean(), disease_a['lean_Rate'].mean()]
#     df['Lean STD'] = [normal['lean_Rate'].std(), nd['lean_Rate'].std(), disease_b['lean_Rate'].std(), disease_a['lean_Rate'].std()]
#     f_statistic3, p_value3 = st.f_oneway(normal['lean_Rate'], nd['lean_Rate'], disease_b['lean_Rate'].dropna(), disease_a['lean_Rate'].dropna() )
#     df['Lean statisctis'] = [p_value3, 0,0,0]

#     df['BMI Mean'] = [normal['BMI'].mean(), nd['BMI'].mean(), disease_b['BMI'].mean(), disease_a['BMI'].mean()]
#     df['BMI STD'] = [normal['BMI'].std(), nd['BMI'].std(), disease_b['BMI'].std(), disease_a['BMI'].std()]
#     _, p_value4 = st.f_oneway(normal['BMI'].dropna(), nd['BMI'].dropna(), disease_b['BMI'].dropna(), disease_a['BMI'].dropna() )
#     df['BMI statisctis'] = [p_value4, 0,0,0]

#     df['Waist Circumference Mean'] = [normal['Waist circumference'].mean(), nd['Waist circumference'].mean(), disease_b['Waist circumference'].mean(), disease_a['Waist circumference'].mean()]
#     df['Waist Circumference STD'] = [normal['Waist circumference'].std(), nd['Waist circumference'].std(), disease_b['Waist circumference'].std(), disease_a['Waist circumference'].std()]
#     _, p_value5 = st.f_oneway(normal['Waist circumference'].dropna(), nd['Waist circumference'].dropna(), disease_b['Waist circumference'].dropna(), disease_a['Waist circumference'].dropna() )
#     df['Waist Circumference statisctis'] = [p_value5, 0,0,0]
    
#     df['Hip Circumference Mean'] = [normal['Hip circumference'].mean(), nd['Hip circumference'].mean(), disease_b['Hip circumference'].mean(), disease_a['Hip circumference'].mean()]
#     df['Hip Circumference STD'] = [normal['Hip circumference'].std(), nd['Hip circumference'].std(), disease_b['Hip circumference'].std(), disease_a['Hip circumference'].std()]
#     _, p_value4 = st.f_oneway(normal['Hip circumference'].dropna(), nd['Hip circumference'].dropna(), disease_b['Hip circumference'].dropna(), disease_a['Hip circumference'].dropna() )
#     df['Hip Circumference statisctis'] = [p_value4, 0,0,0]

#     df['T2d-before-after-0'] = [0,0,len(disease_b[disease_b['T2D Label']=='Before']), len(disease_a[disease_a['T2D Label']=='After'])]
#     df['ASCVD-before-after-0'] = [0,0,len(disease_b[disease_b['ASCVD Label']=='Before']), len(disease_a[disease_a['ASCVD Label']=='After'])]
#     df['MACE-before-after-0'] = [0,0,len(disease_b[disease_b['MACE Label']=='Before']), len(disease_a[disease_a['MACE Label']=='After'])]
#     df['Hypertension-before-after-0'] = [0,0,len(disease_b[disease_b['Hypertension Label']=='Before']), len(disease_a[disease_a['Hypertension Label']=='After'])]
#     return df.round(3)

In [ ]:
# f_tabel = table_generator(label_f)
# f_tabel

In [ ]:
# def new_format(m_tabel):
#     new_tabel = pd.DataFrame()
#     new_tabel['N'] = m_tabel['n']
#     new_tabel['Age'] = m_tabel[['Age Mean', 'Age STD']].astype(str).agg(', '.join, axis=1)
#     new_tabel['VAT'] = m_tabel[['VAT Mean', 'VAT STD']].astype(str).agg(', '.join, axis=1)
#     new_tabel['VAT statisctis'] = m_tabel['VAT statisctis']
#     new_tabel['Lean'] = m_tabel[['Lean Mean', 'Lean STD']].astype(str).agg(', '.join, axis=1)
#     new_tabel['Lean statisctis'] = m_tabel['Lean statisctis']
#     new_tabel['BMI'] = m_tabel[['BMI Mean', 'BMI STD']].astype(str).agg(', '.join, axis=1)
#     new_tabel['BMI statisctis'] = m_tabel['BMI statisctis']
#     new_tabel['Waist Circumference'] = m_tabel[['Waist Circumference Mean', 'Waist Circumference STD']].astype(str).agg(', '.join, axis=1)
#     new_tabel['Waist Circumference statisctis'] = m_tabel['Waist Circumference statisctis']
#     new_tabel['Hip Circumference'] = m_tabel[['Hip Circumference Mean', 'Hip Circumference STD']].astype(str).agg(', '.join, axis=1)
#     new_tabel['Hip Circumference statisctis'] = m_tabel['Hip Circumference statisctis']
#     new_tabel['T2D'] = m_tabel['T2d-before-after-0']
#     new_tabel['ASCVD'] = m_tabel['ASCVD-before-after-0']
#     new_tabel['MACE'] = m_tabel['MACE-before-after-0']
#     new_tabel['Hypertension'] = m_tabel['Hypertension-before-after-0']
#     return new_tabel



In [ ]:
# m_tabel = table_generator(label_m)
# m_tabel = new_format(m_tabel)
# # m_tabel.T.to_csv('/home/jielian/DXA_BA/results/analysis/table1_male.csv')

In [ ]:
# f_tabel = table_generator(label_f)
# f_tabel = new_format(f_tabel)
# # f_tabel.T.to_csv('/home/jielian/DXA_BA/results/analysis/table1_female.csv')

In [ ]:
def bootstrap_confidence_interval(y_true, y_pred, func, alpha =0.95):
    y_true = list(y_true)
    y_pred = list(y_pred)
    n = len(y_true)
    loss = []
    for i in range(n):
        l = func([y_true[i]], [y_pred[i]])
        # print(l)
        loss.append(l)
    a, b =st.norm.interval(alpha=alpha,  loc=np.mean(loss), scale=st.sem(loss))
    if pd.isna(a) or pd.isna(n):
        return a, b
    else:
        return round(a,3), round(b,3)

Aging inference

In [ ]:
def metric_generate(label_f, hc_test_f):
    normal_f = label_f[label_f['Participant ID'].isin(hc_test_f)]
    disease_b = label_f[label_f['BC_label'].isin(['Pre-existing Disease', 'Post-DXA and Pre-Disease'])] 
    disease_a = label_f[label_f['BC_label'].isin(['Post-DXA Disease','Post-DXA and Pre-Disease'])] 
    
    vat_l = label_f[label_f['VAT_label']== 'Hypernormal']
    vat_u = label_f[label_f['VAT_label']== 'Suboptimal']
    print('test: ',len(normal_f))
    print('Pre-existing: ',len(disease_b))
    print('Post-DXA: ', len(disease_a))
    print('Hypernormal: ',len(vat_l))
    print('Suboptimal: ', len(vat_u))
    t40 = normal_f[normal_f['Age_Label']=='40-49']
    db40 = disease_b[disease_b['Age_Label']=='40-49']
    da40 = disease_a[disease_a['Age_Label']=='40-49']
    u40 = vat_u[vat_u['Age_Label']=='40-49']
    l40 = vat_l[vat_l['Age_Label']=='40-49']
    t50 = normal_f[normal_f['Age_Label']=='50-59']
    db50 = disease_b[disease_b['Age_Label']=='50-59']
    da50 = disease_a[disease_a['Age_Label']=='50-59']
    u50 = vat_u[vat_u['Age_Label']=='50-59']
    l50 = vat_l[vat_l['Age_Label']=='50-59']

    t60 = normal_f[normal_f['Age_Label']=='60-69']
    db60 = disease_b[disease_b['Age_Label']=='60-69']
    da60 = disease_a[disease_a['Age_Label']=='60-69']
    u60 = vat_u[vat_u['Age_Label']=='60-69']
    l60 = vat_l[vat_l['Age_Label']=='60-69']

    t70 = normal_f[normal_f['Age_Label']=='70-79']
    db70 = disease_b[disease_b['Age_Label']=='70-79']
    da70 = disease_a[disease_a['Age_Label']=='70-79']
    u70 = vat_u[vat_u['Age_Label']=='70-79']
    l70 = vat_l[vat_l['Age_Label']=='70-79']

    perm=pd.DataFrame()
    perm['Age Group'] = ['Overall','40-49', '50-59', '60-69','70-79']
    perm['Normal Reference Test MAE'] = [mean_absolute_error(normal_f['CA'], normal_f['C_BA']), 
                mean_absolute_error(t40['CA'], t40['C_BA']), 
                mean_absolute_error(t50['CA'], t50['C_BA']),
                mean_absolute_error(t60['CA'], t60['C_BA']),
                mean_absolute_error(t70['CA'], t70['C_BA'])]
    perm['Normal Reference Test MAE CI'] = [bootstrap_confidence_interval(normal_f['CA'], normal_f['C_BA'], mean_absolute_error), 
                bootstrap_confidence_interval(t40['CA'], t40['C_BA'], mean_absolute_error), 
                bootstrap_confidence_interval(t50['CA'], t50['C_BA'], mean_absolute_error),
                bootstrap_confidence_interval(t60['CA'], t60['C_BA'], mean_absolute_error),
                bootstrap_confidence_interval(t70['CA'], t70['C_BA'], mean_absolute_error)]
    perm['Pre-existing disease MAE'] = [mean_absolute_error(disease_b['CA'], disease_b['C_BA']), 
                mean_absolute_error(db40['CA'], db40['C_BA']), 
                mean_absolute_error(db50['CA'], db50['C_BA']),
                mean_absolute_error(db60['CA'], db60['C_BA']),
                mean_absolute_error(db70['CA'], db70['C_BA'])]
    perm['Pre-existing disease MAE CI'] = [bootstrap_confidence_interval(disease_b['CA'], disease_b['C_BA'], mean_absolute_error), 
                bootstrap_confidence_interval(db40['CA'], db40['C_BA'], mean_absolute_error), 
                bootstrap_confidence_interval(db50['CA'], db50['C_BA'], mean_absolute_error),
                bootstrap_confidence_interval(db60['CA'], db60['C_BA'], mean_absolute_error),
                bootstrap_confidence_interval(db70['CA'], db70['C_BA'], mean_absolute_error)]
    perm['Post-DXA disease MAE'] = [mean_absolute_error(disease_a['CA'], disease_a['C_BA']), 
                mean_absolute_error(da40['CA'], da40['C_BA']), 
                mean_absolute_error(da50['CA'], da50['C_BA']),
                mean_absolute_error(da60['CA'], da60['C_BA']),
                mean_absolute_error(da70['CA'], da70['C_BA'])]
    perm['Post-DXA disease MAE CI'] = [bootstrap_confidence_interval(disease_a['CA'], disease_a['C_BA'], mean_absolute_error), 
                bootstrap_confidence_interval(da40['CA'], da40['C_BA'], mean_absolute_error), 
                bootstrap_confidence_interval(da50['CA'], da50['C_BA'], mean_absolute_error),
                bootstrap_confidence_interval(da60['CA'], da60['C_BA'], mean_absolute_error),
                bootstrap_confidence_interval(da70['CA'], da70['C_BA'], mean_absolute_error)]
    perm['Hypernormal MAE'] = [mean_absolute_error(vat_l['CA'], vat_l['C_BA']), 
                mean_absolute_error(l40['CA'], l40['C_BA']), 
                mean_absolute_error(l50['CA'], l50['C_BA']),
                mean_absolute_error(l60['CA'], l60['C_BA']),
                mean_absolute_error(l70['CA'], l70['C_BA'])]
    perm['Hypernormal MAE CI'] = [bootstrap_confidence_interval(vat_l['CA'], vat_l['C_BA'], mean_absolute_error), 
                bootstrap_confidence_interval(l40['CA'], l40['C_BA'], mean_absolute_error), 
                bootstrap_confidence_interval(l50['CA'], l50['C_BA'], mean_absolute_error),
                bootstrap_confidence_interval(l60['CA'], l60['C_BA'], mean_absolute_error),
                bootstrap_confidence_interval(l70['CA'], l70['C_BA'], mean_absolute_error)]

    perm['Suboptimal MAE'] = [mean_absolute_error(vat_u['CA'], vat_u['C_BA']), 
                mean_absolute_error(u40['CA'], u40['C_BA']), 
                mean_absolute_error(u50['CA'], u50['C_BA']),
                mean_absolute_error(u60['CA'], u60['C_BA']),
                mean_absolute_error(u70['CA'], u70['C_BA'])]
    perm['Suboptimal MAE CI'] = [bootstrap_confidence_interval(vat_u['CA'], vat_u['C_BA'], mean_absolute_error), 
                bootstrap_confidence_interval(u40['CA'], u40['C_BA'], mean_absolute_error), 
                bootstrap_confidence_interval(u50['CA'], u50['C_BA'], mean_absolute_error),
                bootstrap_confidence_interval(u60['CA'], u60['C_BA'], mean_absolute_error),
                bootstrap_confidence_interval(u70['CA'], u70['C_BA'], mean_absolute_error)]


    perm['Normal Reference Test MAPE'] = [mean_absolute_percentage_error(normal_f['CA'], normal_f['C_BA']),
                mean_absolute_percentage_error(t40['CA'], t40['C_BA']), 
                mean_absolute_percentage_error(t50['CA'], t50['C_BA']),
                mean_absolute_percentage_error(t60['CA'], t60['C_BA']),
                mean_absolute_percentage_error(t70['CA'], t70['C_BA'])]
    perm['Normal Reference Test MAPE CI'] = [bootstrap_confidence_interval(normal_f['CA'], normal_f['C_BA'], mean_absolute_percentage_error),
                bootstrap_confidence_interval(t40['CA'], t40['C_BA'], mean_absolute_percentage_error), 
                bootstrap_confidence_interval(t50['CA'], t50['C_BA'], mean_absolute_percentage_error),
                bootstrap_confidence_interval(t60['CA'], t60['C_BA'], mean_absolute_percentage_error),
                bootstrap_confidence_interval(t70['CA'], t70['C_BA'], mean_absolute_percentage_error)]
    perm['Pre-existing disease MAPE'] = [mean_absolute_percentage_error(disease_b['CA'], disease_b['C_BA']), 
                mean_absolute_percentage_error(db40['CA'], db40['C_BA']), 
                mean_absolute_percentage_error(db50['CA'], db50['C_BA']),
                mean_absolute_percentage_error(db60['CA'], db60['C_BA']),
                mean_absolute_percentage_error(db70['CA'], db70['C_BA'])]
    perm['Pre-existing disease MAPE CI'] = [bootstrap_confidence_interval(disease_b['CA'], disease_b['C_BA'], mean_absolute_percentage_error), 
                bootstrap_confidence_interval(db40['CA'], db40['C_BA'], mean_absolute_percentage_error), 
                bootstrap_confidence_interval(db50['CA'], db50['C_BA'], mean_absolute_percentage_error),
                bootstrap_confidence_interval(db60['CA'], db60['C_BA'], mean_absolute_percentage_error),
                bootstrap_confidence_interval(db70['CA'], db70['C_BA'], mean_absolute_percentage_error)]    
    perm['Post-DXA disease MAPE'] = [mean_absolute_percentage_error(disease_a['CA'], disease_a['C_BA']), 
                mean_absolute_percentage_error(da40['CA'], da40['C_BA']), 
                mean_absolute_percentage_error(da50['CA'], da50['C_BA']),
                mean_absolute_percentage_error(da60['CA'], da60['C_BA']),
                mean_absolute_percentage_error(da70['CA'], da70['C_BA'])]
    perm['Post-DXA disease MAPE CI'] = [bootstrap_confidence_interval(disease_a['CA'], disease_a['C_BA'], mean_absolute_percentage_error), 
                bootstrap_confidence_interval(da40['CA'], da40['C_BA'], mean_absolute_percentage_error), 
                bootstrap_confidence_interval(da50['CA'], da50['C_BA'], mean_absolute_percentage_error),
                bootstrap_confidence_interval(da60['CA'], da60['C_BA'], mean_absolute_percentage_error),
                bootstrap_confidence_interval(da70['CA'], da70['C_BA'], mean_absolute_percentage_error)]   
    perm['Hypernormal MAPE'] = [mean_absolute_percentage_error(vat_l['CA'], vat_l['C_BA']), 
                mean_absolute_percentage_error(l40['CA'], l40['C_BA']), 
                mean_absolute_percentage_error(l50['CA'], l50['C_BA']),
                mean_absolute_percentage_error(l60['CA'], l60['C_BA']),
                mean_absolute_percentage_error(l70['CA'], l70['C_BA'])]
    perm['Hypernormal MAPE CI'] = [bootstrap_confidence_interval(vat_l['CA'], vat_l['C_BA'], mean_absolute_percentage_error), 
                bootstrap_confidence_interval(l40['CA'], l40['C_BA'], mean_absolute_percentage_error), 
                bootstrap_confidence_interval(l50['CA'], l50['C_BA'], mean_absolute_percentage_error),
                bootstrap_confidence_interval(l60['CA'], l60['C_BA'], mean_absolute_percentage_error),
                bootstrap_confidence_interval(l70['CA'], l70['C_BA'], mean_absolute_percentage_error)]

    perm['Suboptimal MAPE'] = [mean_absolute_percentage_error(vat_u['CA'], vat_u['C_BA']), 
                mean_absolute_percentage_error(u40['CA'], u40['C_BA']), 
                mean_absolute_percentage_error(u50['CA'], u50['C_BA']),
                mean_absolute_percentage_error(u60['CA'], u60['C_BA']),
                mean_absolute_percentage_error(u70['CA'], u70['C_BA'])]
    perm['Suboptimal MAPE CI'] = [bootstrap_confidence_interval(vat_u['CA'], vat_u['C_BA'], mean_absolute_percentage_error), 
                bootstrap_confidence_interval(u40['CA'], u40['C_BA'], mean_absolute_percentage_error), 
                bootstrap_confidence_interval(u50['CA'], u50['C_BA'], mean_absolute_percentage_error),
                bootstrap_confidence_interval(u60['CA'], u60['C_BA'], mean_absolute_percentage_error),
                bootstrap_confidence_interval(u70['CA'], u70['C_BA'], mean_absolute_percentage_error)]    
    
    
    perm['Normal Reference Test MSE'] = [mean_squared_error(normal_f['CA'], normal_f['C_BA']), 
                mean_squared_error(t40['CA'], t40['C_BA']), 
                mean_squared_error(t50['CA'], t50['C_BA']),
                mean_squared_error(t60['CA'], t60['C_BA']),
                mean_squared_error(t70['CA'], t70['C_BA'])]
    perm['Normal Reference Test MSE CI'] = [bootstrap_confidence_interval(normal_f['CA'], normal_f['C_BA'], mean_squared_error), 
                bootstrap_confidence_interval(t40['CA'], t40['C_BA'], mean_squared_error), 
                bootstrap_confidence_interval(t50['CA'], t50['C_BA'], mean_squared_error),
                bootstrap_confidence_interval(t60['CA'], t60['C_BA'], mean_squared_error),
                bootstrap_confidence_interval(t70['CA'], t70['C_BA'], mean_squared_error)]
    perm['Pre-existing disease MSE'] = [mean_squared_error(disease_b['CA'], disease_b['C_BA']), 
                mean_squared_error(db40['CA'], db40['C_BA']), 
                mean_squared_error(db50['CA'], db50['C_BA']),
                mean_squared_error(db60['CA'], db60['C_BA']),
                mean_squared_error(db70['CA'], db70['C_BA'])]
    perm['Pre-existing disease MSE CI'] = [bootstrap_confidence_interval(disease_b['CA'], disease_b['C_BA'], mean_squared_error), 
                bootstrap_confidence_interval(db40['CA'], db40['C_BA'], mean_squared_error), 
                bootstrap_confidence_interval(db50['CA'], db50['C_BA'], mean_squared_error),
                bootstrap_confidence_interval(db60['CA'], db60['C_BA'], mean_squared_error),
                bootstrap_confidence_interval(db70['CA'], db70['C_BA'], mean_squared_error)]   
    perm['Post-DXA disease MSE'] = [mean_squared_error(disease_a['CA'], disease_a['C_BA']), 
                mean_squared_error(da40['CA'], da40['C_BA']), 
                mean_squared_error(da50['CA'], da50['C_BA']),
                mean_squared_error(da60['CA'], da60['C_BA']),
                mean_squared_error(da70['CA'], da70['C_BA'])]
    perm['Post-DXA disease MSE CI'] = [bootstrap_confidence_interval(disease_a['CA'], disease_a['C_BA'], mean_squared_error), 
                bootstrap_confidence_interval(da40['CA'], da40['C_BA'], mean_squared_error), 
                bootstrap_confidence_interval(da50['CA'], da50['C_BA'], mean_squared_error),
                bootstrap_confidence_interval(da60['CA'], da60['C_BA'], mean_squared_error),
                bootstrap_confidence_interval(da70['CA'], da70['C_BA'], mean_squared_error)]   
    perm['Hypernormal MSE'] = [mean_squared_error(vat_l['CA'], vat_l['C_BA']), 
                mean_squared_error(l40['CA'], l40['C_BA']), 
                mean_squared_error(l50['CA'], l50['C_BA']),
                mean_squared_error(l60['CA'], l60['C_BA']),
                mean_squared_error(l70['CA'], l70['C_BA'])]
    perm['Hypernormal MSE CI'] = [bootstrap_confidence_interval(vat_l['CA'], vat_l['C_BA'], mean_squared_error), 
                bootstrap_confidence_interval(l40['CA'], l40['C_BA'], mean_squared_error), 
                bootstrap_confidence_interval(l50['CA'], l50['C_BA'], mean_squared_error),
                bootstrap_confidence_interval(l60['CA'], l60['C_BA'], mean_squared_error),
                bootstrap_confidence_interval(l70['CA'], l70['C_BA'], mean_squared_error)]
    perm['Suboptimal MSE'] = [mean_squared_error(vat_u['CA'], vat_u['C_BA']), 
                mean_squared_error(u40['CA'], u40['C_BA']), 
                mean_squared_error(u50['CA'], u50['C_BA']),
                mean_squared_error(u60['CA'], u60['C_BA']),
                mean_squared_error(u70['CA'], u70['C_BA'])]
    perm['Suboptimal MSE CI'] = [bootstrap_confidence_interval(vat_u['CA'], vat_u['C_BA'], mean_squared_error), 
                bootstrap_confidence_interval(u40['CA'], u40['C_BA'], mean_squared_error), 
                bootstrap_confidence_interval(u50['CA'], u50['C_BA'], mean_squared_error),
                bootstrap_confidence_interval(u60['CA'], u60['C_BA'], mean_squared_error),
                bootstrap_confidence_interval(u70['CA'], u70['C_BA'], mean_squared_error)]  
    perm['Normal Reference Test R2'] = [r2_score(normal_f['CA'], normal_f['C_BA']), 
                r2_score(t40['CA'], t40['C_BA']), 
                r2_score(t50['CA'], t50['C_BA']),
                r2_score(t60['CA'], t60['C_BA']),
                r2_score(t70['CA'], t70['C_BA'])]
    perm['Normal Reference Test R2 CI'] = [bootstrap_confidence_interval(normal_f['CA'], normal_f['C_BA'], r2_score), 
                bootstrap_confidence_interval(t40['CA'], t40['C_BA'], r2_score), 
                bootstrap_confidence_interval(t50['CA'], t50['C_BA'], r2_score),
                bootstrap_confidence_interval(t60['CA'], t60['C_BA'], r2_score),
                bootstrap_confidence_interval(t70['CA'], t70['C_BA'], r2_score)]
    perm['Pre-existing disease R2'] = [r2_score(disease_b['CA'], disease_b['C_BA']), 
                r2_score(db40['CA'], db40['C_BA']), 
                r2_score(db50['CA'], db50['C_BA']),
                r2_score(db60['CA'], db60['C_BA']),
                r2_score(db70['CA'], db70['C_BA'])]
    perm['Pre-existing disease R2 CI'] = [bootstrap_confidence_interval(disease_b['CA'], disease_b['C_BA'], r2_score), 
                bootstrap_confidence_interval(db40['CA'], db40['C_BA'], r2_score), 
                bootstrap_confidence_interval(db50['CA'], db50['C_BA'], r2_score),
                bootstrap_confidence_interval(db60['CA'], db60['C_BA'], r2_score),
                bootstrap_confidence_interval(db70['CA'], db70['C_BA'], r2_score)]
    perm['Post-DXA disease R2'] = [r2_score(disease_a['CA'], disease_a['C_BA']), 
                r2_score(da40['CA'], da40['C_BA']), 
                r2_score(da50['CA'], da50['C_BA']),
                r2_score(da60['CA'], da60['C_BA']),
                r2_score(da70['CA'], da70['C_BA'])]
    perm['Post-DXA disease R2 CI'] = [bootstrap_confidence_interval(disease_a['CA'], disease_a['C_BA'], r2_score), 
                bootstrap_confidence_interval(da40['CA'], da40['C_BA'], r2_score), 
                bootstrap_confidence_interval(da50['CA'], da50['C_BA'], r2_score),
                bootstrap_confidence_interval(da60['CA'], da60['C_BA'], r2_score),
                bootstrap_confidence_interval(da70['CA'], da70['C_BA'], r2_score)]                
    perm['Hypernormal R2'] = [r2_score(vat_l['CA'], vat_l['C_BA']), 
                r2_score(l40['CA'], l40['C_BA']), 
                r2_score(l50['CA'], l50['C_BA']),
                r2_score(l60['CA'], l60['C_BA']),
                r2_score(l70['CA'], l70['C_BA'])]
    perm['Hypernormal R2 CI'] = [bootstrap_confidence_interval(vat_l['CA'], vat_l['C_BA'], r2_score), 
                bootstrap_confidence_interval(l40['CA'], l40['C_BA'], r2_score), 
                bootstrap_confidence_interval(l50['CA'], l50['C_BA'], r2_score),
                bootstrap_confidence_interval(l60['CA'], l60['C_BA'], r2_score),
                bootstrap_confidence_interval(l70['CA'], l70['C_BA'], r2_score)]
    perm['Suboptimal R2'] = [r2_score(vat_u['CA'], vat_u['C_BA']), 
                r2_score(u40['CA'], u40['C_BA']), 
                r2_score(u50['CA'], u50['C_BA']),
                r2_score(u60['CA'], u60['C_BA']),
                r2_score(u70['CA'], u70['C_BA'])]
    perm['Suboptimal R2 CI'] = [bootstrap_confidence_interval(vat_u['CA'], vat_u['C_BA'], r2_score), 
                bootstrap_confidence_interval(u40['CA'], u40['C_BA'], r2_score), 
                bootstrap_confidence_interval(u50['CA'], u50['C_BA'], r2_score),
                bootstrap_confidence_interval(u60['CA'], u60['C_BA'], r2_score),
                bootstrap_confidence_interval(u70['CA'], u70['C_BA'], r2_score)] 
    perm = perm.round(3)
    return perm

In [ ]:
def new_metric_gene(perm):
    new_metric = pd.DataFrame()
    new_metric['Age Group'] = perm['Age Group']
    new_metric['Normal Reference Test MAE'] = perm[['Normal Reference Test MAE', 'Normal Reference Test MAE CI']].astype(str).agg(', '.join, axis=1)
    new_metric['Hypernormal MAE'] = perm[['Hypernormal MAE', 'Hypernormal MAE CI']].astype(str).agg(', '.join, axis=1)
    new_metric['Suboptimal MAE'] = perm[['Suboptimal MAE', 'Suboptimal MAE CI']].astype(str).agg(', '.join, axis=1)
    new_metric['Pre-existing Disease MAE'] = perm[['Pre-existing disease MAE', 'Pre-existing disease MAE CI']].astype(str).agg(', '.join, axis=1)
    new_metric['Post-DXA Disease MAE'] = perm[['Post-DXA disease MAE', 'Post-DXA disease MAE CI']].astype(str).agg(', '.join, axis=1)
    
    new_metric['Normal Reference Test MAPE'] = perm[['Normal Reference Test MAPE', 'Normal Reference Test MAPE CI']].astype(str).agg(', '.join, axis=1)
    new_metric['Hypernormal MAPE'] = perm[['Hypernormal MAPE', 'Hypernormal MAPE CI']].astype(str).agg(', '.join, axis=1)
    new_metric['Suboptimal MAPE'] = perm[['Suboptimal MAPE', 'Suboptimal MAPE CI']].astype(str).agg(', '.join, axis=1)
    new_metric['Pre-existing Disease MAPE'] = perm[['Pre-existing disease MAPE', 'Pre-existing disease MAPE CI']].astype(str).agg(', '.join, axis=1)
    new_metric['Post-DXA Disease MAPE'] = perm[['Post-DXA disease MAPE', 'Post-DXA disease MAPE CI']].astype(str).agg(', '.join, axis=1)

    new_metric['Normal Reference Test MSE'] = perm[['Normal Reference Test MSE', 'Normal Reference Test MSE CI']].astype(str).agg(', '.join, axis=1)
    new_metric['Hypernormal MSE'] = perm[['Hypernormal MSE', 'Hypernormal MSE CI']].astype(str).agg(', '.join, axis=1)
    new_metric['Suboptimal MSE'] = perm[['Suboptimal MSE', 'Suboptimal MSE CI']].astype(str).agg(', '.join, axis=1)
    new_metric['Pre-existing Disease MSE'] = perm[['Pre-existing disease MSE', 'Pre-existing disease MSE CI']].astype(str).agg(', '.join, axis=1)
    new_metric['Post-DXA Disease MSE'] = perm[['Post-DXA disease MSE', 'Post-DXA disease MSE CI']].astype(str).agg(', '.join, axis=1)

    new_metric['Normal Reference Test R2'] = perm['Normal Reference Test R2']
    new_metric['Hypernormal  R2'] = perm['Hypernormal R2']
    new_metric['Suboptimal R2'] = perm['Suboptimal R2']
    new_metric['Pre-existing Disease R2'] = perm['Pre-existing disease R2']
    new_metric['Post-DXA Disease R2'] = perm['Post-DXA disease R2']
    return new_metric.T

In [ ]:
f_metrics = metric_generate(label_f, hc_test_f)
f_metrics = new_metric_gene(f_metrics)
print('Female')
print(f_metrics)


In [ ]:
m_metrics = metric_generate(label_m, hc_test_m)
m_metrics = new_metric_gene(m_metrics)
print('Male')
print(m_metrics)


In [ ]:
label_f['BA-CA'] = label_f['C_BA'] - label_f['CA']
label_m['BA-CA'] = label_m['C_BA'] - label_m['CA']

In [ ]:

def s_plot(label_f, hc_test_f, save_path=''):
    normal_f = label_f[label_f['Participant ID'].isin(hc_test_f)]
    disease_b = label_f[label_f['BC_label'].isin(['Pre-existing Disease', 'Post-DXA and Pre-Disease'])] 
    disease_a = label_f[label_f['BC_label'].isin(['Post-DXA Disease','Post-DXA and Pre-Disease'])] 
    vat_l = label_f[label_f['VAT_label']== 'Hypernormal']
    vat_u = label_f[label_f['VAT_label']== 'Suboptimal']

    xt = np.array(normal_f['CA'])
    yt = np.array(normal_f['BA-CA'])
    color1 = np.random.rand(len(xt))

    xh = np.array(vat_l['CA'])
    yh = np.array(vat_l['BA-CA'])
    color2 = np.random.rand(len(xh))

    xs = np.array(vat_u['CA'])
    ys = np.array(vat_u['BA-CA'])
    color3 = np.random.rand(len(xs))

    xdb = np.array(disease_b['CA'])
    ydb = np.array(disease_b['BA-CA'])
    color4 = np.random.rand(len(xdb))

    xda = np.array(disease_a['CA'])
    yda = np.array(disease_a['BA-CA'])
    color5 = np.random.rand(len(xda))

    fig, ax = plt.subplots(1,5, figsize=(30, 5))
    ax[0].scatter(x = xt, y = yt,alpha =0.5)
    ax[0].axhline(y = np.mean(yt), color = 'r', linestyle = '-')
    ax[0].axhline(y = np.quantile(yt, 0.25), color = 'b', linestyle = '-')
    ax[0].axhline(y = np.quantile(yt, 0.75), color = 'b', linestyle = '-')
    ax[0].set_xlabel("CA")
    ax[0].set_ylabel("BCBA-CA")
    ax[0].title.set_text('Normal Reference Group')
    ax[0].set_xlim([40, 80])
    ax[0].set_ylim([-5, 5])

    ax[1].scatter(x = xs, y = ys,alpha =0.5)
    ax[1].axhline(y = np.mean(ys), color = 'r', linestyle = '-')
    ax[1].axhline(y = np.quantile(ys, 0.25), color = 'b', linestyle = '-')
    ax[1].axhline(y = np.quantile(ys, 0.75), color = 'b', linestyle = '-')
    ax[1].set_xlabel("CA")
    ax[1].set_ylabel("BCBA-CA")
    ax[1].title.set_text('Suboptimal')
    ax[1].set_xlim([40, 80])
    ax[1].set_ylim([-5, 5])

    ax[2].scatter(x = xh, y = yh,alpha =0.5)
    ax[2].axhline(y = np.mean(yh), color = 'r', linestyle = '-')
    ax[2].axhline(y = np.quantile(yh, 0.25), color = 'b', linestyle = '-')
    ax[2].axhline(y = np.quantile(yh, 0.75), color = 'b', linestyle = '-')
    ax[2].set_xlabel("CA")
    ax[2].set_ylabel("BCBA-CA")
    ax[2].title.set_text('Hypernormal')
    ax[2].set_xlim([40, 80])
    ax[2].set_ylim([-5, 5])

    ax[3].scatter(x = xda, y = yda,alpha =0.5)
    ax[3].axhline(y = np.mean(yda), color = 'r', linestyle = '-')
    ax[3].axhline(y = np.quantile(yda, 0.25), color = 'b', linestyle = '-')
    ax[3].axhline(y = np.quantile(yda, 0.75), color = 'b', linestyle = '-')
    ax[3].set_xlabel("CA")
    ax[3].set_ylabel("BCBA-CA")
    ax[3].title.set_text('Post-DXA Disease Group')
    ax[3].set_xlim([40, 80])
    ax[3].set_ylim([-5, 5])

    ax[4].scatter(x = xdb, y = ydb,alpha =0.5)
    ax[4].axhline(y = np.mean(ydb), color = 'r', linestyle = '-')
    ax[4].axhline(y = np.quantile(ydb, 0.25), color = 'b', linestyle = '-')
    ax[4].axhline(y = np.quantile(ydb, 0.75), color = 'b', linestyle = '-')
    ax[4].set_xlabel("CA")
    ax[4].set_ylabel("BCBA-CA")
    ax[4].title.set_text('Pre-existing Group')
    ax[4].set_xlim([40, 80])
    ax[4].set_ylim([-5, 5])


    if len(save_path) >5:
        plt.savefig(save_path)
    plt.show()

In [ ]:
s_plot(label_f, hc_test_f, 'results/analysis/female_scot.png')
s_plot(label_f, hc_test_f, 'results/analysis/male_scot.png')